# 07: Go capabilities: Intro to testing

## Objectives
+ testing approaches and conventions


## Summary

Go provides a lot of built-in capabilities for testing your code.

As a convention, Go test files should end in `<name>_test.go`.

In the case that we're testing from the user's PoV we should test only what is exposed (that is the uppercase functions) and by convention these tests should go on a `<package-name>_test` package. Alternatively, if we want to test also the unexposed functions the test file should be in the same package as the source and we could use the name `<xyz>_internal_test.go` name.

## Testing that a function writes something to stdout

Go provides a specific tool based on a test function's name which can be used to test that what a function sends to the standard output is according to your expectations.

In order to use use, in your test file, you have to define a function named `Example_<func-name-to-test>()` and use the following syntax including comments:

```go
// Checking main output to stdout
func Example_main() {
  main()
  // Output:
  // Hello world
}
```

### Lab

Write an internal test file for the *"Hello, world!"* program that uses the `Example_<func-name-to-test>()` approach to validate the output is the one expected.

Most of the times, you will need to test functions that return some values. In those cases, the approach will be different from the one used in the previous example.

In those scenarios you need to use the `testing` package.

Those test cases should be named `Test<func-name-to-test>(t *testing.T)`. That function should be used to run one or more tests on a function.

Every test function should feature the usual four steps:
+ arrange: where we set up everything we need to run the test
+ act: where we call the tested function
+ assert: where we check that the actual output meets our expectations. When it doesn't, we can use the `t.ErrorF()` function that lets you craft an error message and signal the execution of the test as failed.
+ teardown: where we clean back and revert the state of the system to how it was before we began the test.


### Lab

Write a program that displays a "Hello, world" message in a `main()` function by delegating the message to a `greet()` function that returns the greeting message.

Then create a `TestGreet()` function that validates that the `greet()` function works as expected. Change the expected error message to see the test fail to familiarize yourself with that situation.

| EXAMPLE: |
| :------- |
| See [02_test-hello-test](02_test-hello-test/) for a runnable example. |

## Test-Driven Design (TDD) in Go

TDD (Test-Driven Design) is a discipline that promotes a development control loop consisting of:

1. Write a test

2. Write code in your program to make the compiler pass

3. Make it fail, and confirm that the error messages help identify the problem.

4. Write enough code to make the test pass

5. Refactor

In the following subsections, we'll create several examples that illustrate certain TDD techniques for Go.

### Setting up shop when doing TDD with Go

When writing test, it's quite common to find repeatable patterns that could be refactored in helper functions.

For example:

```go
func TestHello(t *testing.T) {
  t.Run("sending empty name", t.Run(t *testing.T) {
    want := "Hello, World!"
    got := Hello("")
    if got != want {
      t.Errorf("got %q, but wanted %q", got, want)
    }
  })

  t.Run("sending name", t.Run(t *testing.T) {
    want := "Hello, Jason!"
    got := Hello("Jason")
    if got != want {
      t.Errorf("got %q, but wanted %q", got, want)
    }
  })  
}
```

| NOTE: |
| :---- |
| For now, it's enough to know that `t *testing.T` is your hook into the testing framework that provides convenieve methods such as `t.Errorf`. |

Note how the assertion code is the same in both subtests. In such cases, we can refactor by creating a helper function `assertCorrectMessage`:

```go
func assertCorrectMessage(t testing.TB, got, want string) {
    t.Helper()
    if got != want {
      t.Errorf("got %q, but wanted %q", got, want)
    }  
}
```

For helper functions, it's a good idea to accept a `testing.TB`, which is an interface that both `testing.B` and `testing.T` satisfy, so that you can call helper functions from a test.

The call `t.Helper()` is needed to tell the test suite that this method is a helper. For example, this will report the correct line numbers when the test fails.

#### Lab

Create a Go project for a Hello, world application that delivers the message through a function `Hello` that can receive a named individual and generate either the message "Hello to {{name}}!" or "Hello, World!" depending on whether a name is given to the function or not.

You should create a function that validates what's written on stdout, and another function that validates the different scenarios of the `Hello` function. Write a helper to reduce the boilerplate code used when checking the results.

| EXAMPLE: |
| :------- |
| See [03_tdd-hello-world](03_tdd-hello-world/) for a runnable example. |

### Examples

Go examples are executed just like tests, so that you can have examples that reflect what the code actually does.

Examples reside in a package's test file (e.g., `hello_test.go`) and have the following shape:

```go
func ExampleAdd() {
  sum := Add(1, 5)
  fmt.Println(sum)
  // Output: 6
}
```

### Benchmarks

Benchmarks are another first-class feature of the language, and it is very similar to writing tests.

```go
func BenchmarkRepeat(b *testing.B) {
	for i := 0; i < b.N; i++ {
		iteration.Repeat("a")
	}
}
```

The function is named `BenchmarkXyz(b *testing.B)`.

`testing.B` gives you access to `b.N` and within the benchmark function code, we run `Repeat("a")` *b.N* times and measure how long it takes.

You can run a benchmark test running:

```bash
# when in the directory with the benchmark tests
go test -bench=.

# when in the root of the project
go test <package>/<test_file>_test.go -bench=.
```

#### Labs

Create a test file with benchmarks for a function `Repeat` that concatenates 5 times a string passed as an argument and implemented in three different ways:
+ using a for loop and `+` to concatenate
+ using a `strings.Builder`
+ using `strings.Repeat`

| EXAMPLE: |
| :------- |
| See [04_tdd-benchmarks](04_tdd-benchmarks/) for a runnable example. |

### Variadic functions refresh

Go allows you to pass a variable number of arguments to a function using `...`.

```go
func SumAll(vectorsToSum ...[]int) {
  ...
}
```

#### Labs

Using TDD, write a function `SumAll` which takes a varying number of slices and returns a new slice containing the totals for each slice passed in.

For example:
+ `SumAll([]int{1, 2}, []int{0, 9})` would return `[]int{3, 9}`.
+ `SumAll([]int{1, 1, 1})` would return `[]int{3}`.

Once your tests pass, confirm there you get 100% coverage by running `go test -cover`.

| EXAMPLE: |
| :------- |
| See [05_tdd-cover-variadic-slices](05_tdd-cover-variadic-slices/) for a runnable example. |

### Refresh on methods and interfaces

You can use almost any named typed as a receiver for a method, except for pointers and interfaces.

#### Lab

This lab illustrates some basic ideas on structs, methods, and interfaces using a TDD approach.

The details for the different stages can be found in [06_tdd-structs-methods-interfaces](06_tdd-structs-methods-interfaces/README.md).


### Refresh on pointers

In Go, when you call a function or a method, the arguments are passed by value, and therefore, copied.

That is, when you pass a value to a function or method, or use a value as a method receiver, the function or method gets a copy of the value you're passing. As a result, any modifications you make in the copy won't be reflected on the original value.

Once you need to use a pointer as a receiver in a particular method, it is a common convention to also change the ones that wouldn't require a pointer for consistency.

### Refresh on errors

Methods and functions should return a value of type `error` to signal error conditions found in your application code.

To signal the absence of error, you should return `nil`.

Errors can be created using `errors.New(errMsg string)` or `fmt.Errorf(format str, args ...any)`.

The `func (e error) Error() string` method defined on errors can be used to get the string representation of an error.

When you want to check in a conditional statement for an specific error, you will have to define a *sentinel error*:

```go
// first alias a string to the new type
type conditionErr string

// then implement the Error method
func (e conditionErr) Error() string {
  return string(e)
}

// finally, define your method as a const
const ErrCondition = conditionErr("description of the err")
```

When creating several sentinel errors, you can create a generic alias and use it as a sort of constructor for the different sentinel errors:

```go
type libErr string

func (e libErr) Error() string {
  return string(e)
}

const (
  ErrLibCondition1 = libErr("description for cond 1")
  ErrLibCondition2 = libErr("description for cond 2")
  ...
)
```

| EXAMPLE: |
| :------- |
| See [dictionary-v3](./08_tdd-maps/dictionary-v3/) for a runnable example. |

#### Lab

This lab illustrates the basics of pointers and errors.

The details for the different stages of the project can be found in [07_tdd-pointers-errors](07_tdd-pointers-errors/README.md).

### Refresh on maps

You optionally receive an `ok` signal from a map when trying to retrieve the value associated to a key:

```go
value, ok := myMap[key]
if !ok {
  // key was not present in the map
}
```

Maps (as well as slices), allow their elements to be modified without using a pointer to them. This is because maps (and slices) are actually pointers to more complex underlying data structures. Maps (and slices) surface a very simple interface that provides a better DX than we would have if interacting with the underlying representation.

Maps can be `nil`. A `nil` map behaves like an empty map when accessing it, but if you try to add a new entry into a map you'll get a runtime panic.

As a result, it is considered a good practice to either do:

```go
var m = map[string]string{} // empty map, won't panic when adding items

var m = make(map[string]string, mapLen)
```

Entries can be removed from maps doing:

```go
delete(m, k) // removes the key-value identified by k from the map m
```

Note that delete does not fail if `k` is not found in the map.

#### Lab

This lab illustrates the basics of maps through an example leveraging a TDD approach.

The details for the different stages of the project can be found in [08_tdd-maps](08_tdd-maps/README.md).

### Dependency Injection

Dependency Injection (DI) is a great pattern to use, especially when you are following TDD, as it will facilitate testing by injecting certain dependencies needed by our code.

| NOTE: |
| :---- |
| It is a general misconception to think that you always need a framework when using DI. In its most simple form is just a simple design pattern that helps us with the wiring of the dependencies of a software component. |

> **Dependency Injection** is a pattern in which the dependencies of a component are provided as inputs by an external entity, often referred to as the **injector**.

The main advantage of this approach is improved decoupling, especially for modules depending on stateful instances.

When using DI, each dependency is received from the outside. In practice, it means that the dependent component can be configured to use any compatible dependency, and therefore, the component can be reused in different contexts with minimal effort.


The following diagram describes the elements in play when using this design pattern:

![Dependency Injection](images/dependency-injection.png)

+ A *Service* expects a dependency with a predetermined interface

+ An *Injector* retrieves and creates an actual concrete instance that implements such an interface and passes it (*inject it*) into the *Service*. That is, the *Injector* is responsible for providing an instance that fulfills the *Service* dependency.

There are some disadvantages associated to the *Dependency Injection* pattern.

+ It makes the application more difficult to understand, as dependencies are not resolved at coding time.

    This becomes especially evident in large applications with a big and complex dependency graph.

+ The dependencies must be ready and available when we inject them into the dependent objects, and before we invoke any of the methods of the dependent object that used *DI*.

Another pattern related to *DI* is the **Inversion of Control** pattern.

> The **Inversion of Control** pattern allows you to shift the responsibility of wiring the modules of an application to a 3rd party entity.

This entity can be a **service locator**, that is, a component which will servce a dependency through a method invocation (as in `serviceLocator.get('db')`) or a **dependency injection container**, a system that injects the dependencies into a component based on some metadata specified in code or in a configuration file.

Getting back to our Go world.

Imagine that we want to test a function such as:

```go
func Greet(name string) {
  fmt.Printf("Hello to %s!", name)
}
```

Because `fmt.Print` writes the output to stdout, it makes the function pretty hard to test.

If we could *inject* into the component the dependency of printing so that the function wouldn't depend directly on `fmt.Printf` that will simplify things a lot. In non-testing scenarios, we would still inject the dependency that writes the message to stdout, but we could use a different dependency for our tests.

In particular, for the scenario presented above, we can see that `fmt.Printf` is actually an invocation of `fmt.Fprintf` on which the first argument is `os.Stdout`.

This first argument is an `io.Writer`, which is defined as:

```go
type Writer interface {
  Write(p []byte) (n int, err error)
}
```

That is, `io.Writer` is an interface with a single method `Write` that receives a slice of bytes to write, and returns how many bytes were written and an error.

We can deduce at this point that `os.Stdout` is a value implementing the `Writer` interface.

With all the previous information in place, it seems that it'll be great to have our function defined as:

```go
func Greet(w io.Writer, name string)
```

#### Lab

This lab illustrates the basics of DI.

The details for the different stages of the project can be found in [09_tdd-di](09_tdd-di/README.md).

### Mocking

As it happens with *DI*, it is a misconception that you need a framework to use mocking.

In Go, it is extremely easy to create mocks, stubs, and spies just by leveraging *DI*, interfaces, and methods.

In the [10_tdd-mocking](10_tdd-mocking/README.md) lab you will have a very comprehensive and step-by-step guide about it.

#### Lab

This lab illustrates the basics of mocking in Go. You will be asked to use DI, interfaces, and methods to properly test a simple application.

In the process you will have to create mock implementations for certain areas of your code, so that they can be properly tested.

In the [10_tdd-mocking](10_tdd-mocking/README.md) lab you will have a very comprehensive and step-by-step guide about it.

#### Concurrency

In Go most operations are blocking, meaning that it has to finish before Go handles the next operation.

If you want to implement a non-blocking operation, you need to run it in a separate *process* called *goroutine*.

Using concurrency naively can lead to bugs that are extremely difficult to fix, because they might only show up when certain sequence of calls are give (race conditions).

Go tooling can help you identify those using:

```bash
$ go test -race
==================
WARNING: DATA RACE
Write at 0x00c000016510 by goroutine 9:
  runtime.mapassign_faststr()
.../concurrency-v1/concurrency.go:15 +0x78

Previous write at 0x00c000016510 by goroutine 8:
...
```

Go promotes the use of channels for coordinating routines. Channels are nothing more than concurrent-safe data structures that can both send and receive values.

#### Lab

This lab illustrates the basics and pitfalls of applying concurrency to an existing project. Goroutines, channels, and race conditions are introduced.

You can find all the details in [11_tdd-concurrency](11_tdd-concurrency/README.md)

### Using `select` to synchronize concurrent processes

The `select` statement lets a goroutine wait on multiple communication operations in a clear and succinct way.

A `select` statement blocks until one of its cases run, and then it executes that case. It chooses one at random it multiple are ready:

```go
select {
case val1 <- chanX:
  // ... do something ...
case val2 <- chanY:
  // ... do something else ...
}
```

Aditionally, you can include a `default` case, which will run if no other case is ready.

```go
select {
case i := <-c:  // blocks until something is ready on c
  // ... do something with i ...
default:
  // ... do something if c is blocking ...
}
```

### Lab

This lab illustrates how to use `select` to synchronize multiple goroutines. Additionally, mocking HTTP servers with the `net/http/httptest` package is also introduced.

You can find all the details in [12_tdd-concurrency-select](12_tdd-concurrency-select/README.md)

## Reflection